In [ ]:
!rm -rf output lost+found kernel_meta
!pip uninstall mindnlp mindspore -y
# !git clone https://hf-mirror.com/Kamyar-zeinalipour/TR_QUIZ_GEN_MULTI_LLAMA7B
!pip install mindspore==2.4.0
!rm -rf TR_QUIZ_GEN_MULTI_LLAMA7B/adapter_model.safetensors
!pip uninstall peft transformers -y
!pip install datasets
!pip install git+https://github.com/mindspore-lab/mindnlp.git
!pip install safetensors
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
if not os.path.exists("TR_QUIZ_GEN_MULTI_LLAMA13B/adapter_model.safetensors") and not os.path.exists("TR_QUIZ_GEN_MULTI_LLAMA13B/adapter_model.ckpt"):
    !wget -P TR_QUIZ_GEN_MULTI_LLAMA13B https://hf-mirror.com/Kamyar-zeinalipour/TR_QUIZ_GEN_MULTI_LLAMA13B/resolve/main/adapter_model.safetensors
if not os.path.exists("TR_QUIZ_GEN_MULTI_LLAMA7B/adapter_model.safetensors") and not os.path.exists("TR_QUIZ_GEN_MULTI_LLAMA7B/adapter_model.ckpt"):
    !wget -P TR_QUIZ_GEN_MULTI_LLAMA7B https://hf-mirror.com/Kamyar-zeinalipour/TR_QUIZ_GEN_MULTI_LLAMA7B/resolve/main/adapter_model.safetensors
!pip uninstall soundfile -y
!pip uninstall mindformers -y

Found existing installation: mindspore 2.3.0
Uninstalling mindspore-2.3.0:
  Successfully uninstalled mindspore-2.3.0
Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     |████████████████████████████████| 333.7 MB 74.8 MB/s eta 0:00:01     |███████████████▋                | 163.0 MB 67.4 MB/s eta 0:00:03
     |████████████████████████████████| 450 kB 55.6 MB/s eta 0:00:01
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.
Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     |████████████████████████████████| 484 kB 42.2 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 62.9 MB/s eta 0:00:01
     |████████████████████████████████| 40.5 MB 38.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 36.4 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 53.1 MB/s eta 0:00:01
     |████████████████████████████

In [2]:
#!huggingface-cli login "sgfdoax42skw123" #更换为自己的key
# repokey="wsawqqwe123ser"
!pip show mindspore
!pip show mindnlp
!pip show transformers
!pip show tokenizers

Name: mindspore
Version: 2.4.0
Summary: MindSpore is a new open source deep learning training/inference framework that could be used for mobile, edge and cloud scenarios.
Home-page: https://www.mindspore.cn
Author: The MindSpore Authors
Author-email: contact@mindspore.cn
License: Apache 2.0
Location: /home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages
Requires: astunparse, pillow, packaging, psutil, numpy, safetensors, protobuf, asttokens, scipy
Required-by: mindnlp
Name: mindnlp
Version: 0.4.1
Summary: An open source natural language processing research tool box. Git version: [sha1]:ab20e2e7, [branch]: (HEAD -> master, origin/master, origin/HEAD)
Home-page: https://github.com/mindlab-ai/mindnlp/tree/master/
Author: MindSpore Team
Author-email: None
License: Apache 2.0
Location: /home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages
Requires: sentencepiece, pyctcdecode, evaluate, tqdm, pillow, datasets, mindspore, tokenizers, safetensors, addict, pytest, requ

In [3]:
#访问数据集文件问题
import argparse
import time
import pandas as pd
import mindspore as ms
from mindspore import nn,context
# import mindnlp
# from mindnlp.core import no_grad
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer
from mindnlp.peft import PeftConfig,PeftModel,get_peft_model
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="Ascend", save_graphs=False)
context.set_auto_parallel_context(parallel_mode=ms.ParallelMode.AUTO_PARALLEL)
ms.ms_memory_recycle()

[WARNING] GE_ADPT(1578,ffffb50110b0,python):2025-02-16-17:28:36.745.841 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleGetModelId failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleGetModelId
[WARNING] GE_ADPT(1578,ffffb50110b0,python):2025-02-16-17:28:36.745.892 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleLoadFromMem failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleLoadFromMem
[WARNING] GE_ADPT(1578,ffffb50110b0,python):2025-02-16-17:28:36.745.911 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleUnload failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleUnload
[WARNING] GE_ADPT(1578,ffffb50110b0,python):2025-02-16-17:28:36.746.165 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymA

In [4]:
import gc
!echo "" > result.csv

In [ ]:
df = pd.read_parquet("https://hf-mirror.com//datasets/Kamyar-zeinalipour/Turkish-Quiz-Instruct/data/train-00000-of-00001.parquet")
print(df.info)
from datasets import load_dataset
# from mindnlp.dataset import load_dataset
# ms.dataset.config.set_seed(0)
ds = load_dataset("Kamyar-zeinalipour/Turkish-Quiz-Instruct")
train=ds["train"]
output_file="Train-dataset.csv"
# df_selected=das.source.ds.features[["content","short_questions"]]
# 将数据转换为 DataFrame
df = pd.DataFrame(train)
print(df.columns)
# 选择需要的列
# df_selected = df[["content", "short_questions"]]
df_selected = df[["content", "multiple_questions"]]
# 将 DataFrame 写入 CSV 文件
df_selected.to_csv(output_file, index=False)
print("File process ready")
with open(output_file,"w") as f:
    for ex in train:
        ct=ex["content"]
        mq=ex["multiple_questions"]
        if "right_answer:" in mq:
            mq_parts = mq.split("right_answer:")
            mq = mq_parts[0].strip()  # 取 right_answer 之前的部分，并去除前后空白字符
#         sq=ex["short_questions"]
#         sb=ex["subject"]
#         st=ex["subsubtopic"]
        f.write(f"{ct}\t{mq}\n")
print(ds[0])
#指定 Parquet 文件的路径
# parquet_file_path = 'Turkish-Quiz-Instruct/data/train-00000-of-00001.parquet'

# # 读取 Parquet 文件
# df = pd.read_parquet(parquet_file_path)

# # 查看数据的前几行
# print(df.head())

File process ready


In [ ]:
from safetensors.numpy import load_file
import mindspore as ms
from mindspore.train.serialization import save_checkpoint
import numpy as np
# # # if not os.path.exists("TR_QUIZ_GEN_MULTI_LLAMA13B/adapter_model.ckpt"):
# # # 加载 .safetensors 文件
safetensors_path = "TR_QUIZ_GEN_MULTI_LLAMA7B/adapter_model.safetensors"
state_dict_numpy = load_file(safetensors_path)

# 将 numpy 数组转换为 MindSpore 参数格式
params_list = []
for name, param_np in state_dict_numpy.items():
    # 创建 MindSpore 参数对象
    param_np = param_np.astype(np.float16)
    ms_param = ms.Parameter(ms.Tensor(param_np), name=name)
    params_list.append(ms_param)

# 保存为 .ckpt 文件
ckpt_path = "TR_QUIZ_GEN_MULTI_LLAMA7B/adapter_model.ckpt"
save_checkpoint(params_list, ckpt_path)
ms.ms_memory_recycle()
# !rm -rf TR_QUIZ_GEN_MULTI_LLAMA13B/adapter_model.safetensors
print("Transfer OK")

Transfer OK


In [7]:

# Constants
MAX_NEW_TOKENS = 8
TOP_K = 20
TOP_P = 0.9
NO_REPEAT_NGRAM_SIZE = 4
REPETITION_PENALTY = 1.0

def get_code_completion(prompt: str, model, tokenizer, temperature: float) -> str:
    """Generate code completion for a given prompt"""
    try:
        tokenized_input = tokenizer(prompt, return_tensors="ms")
        input_ids = tokenized_input["input_ids"]
        print("input_ids dtype:", input_ids.dtype)  # 检查是否为 int32/int64
        outputs = model.generate(
            input_ids=tokenized_input["input_ids"],
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=temperature,
            top_k=TOP_K,
            top_p=TOP_P,
            do_sample=True,
            no_repeat_ngram_size=NO_REPEAT_NGRAM_SIZE,
            repetition_penalty=REPETITION_PENALTY,
            output_attentions=False,  # 关闭注意力输出以减少计算
    output_hidden_states=False
        )
#         del tokenized_input
#         del input_ids
#         gc.collect()
        ms.ms_memory_recycle()
        return tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]
    except Exception as e:
#         print(len(str(e)))
        print(f"Error during code generation: {str(e)}")
        raise



In [8]:
from mindnlp.transformers import LlamaForCausalLM, LlamaTokenizer
import mindspore as ms

class SafeLlamaModel(LlamaForCausalLM):  # 继承自 LLaMA 的模型类
    def __init__(self, original_model):
        super().__init__(original_model.config)
        # 复制原模型的参数和子模块
        self.load_state_dict(original_model.parameters_dict())
        self.model = original_model.model  # LLaMA 的主干网络
        self.lm_head = original_model.lm_head  # 输出头

    def construct(self, input_ids, **kwargs):
        # 调用原模型的前向计算
        outputs = self.model(input_ids, **kwargs)
        hidden_states = outputs.last_hidden_state
        logits = self.lm_head(hidden_states)
        # 强制转换 logits 为 float32
        logits = logits.astype(ms.float32)
        return logits  # 返回与原模型一致的输出结构



In [ ]:
from mindnlp.transformers import AutoModel
def load_model(model_path: str) -> tuple:
    """Load the pre-trained model and tokenizer"""
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    print(tokenizer)
    config = PeftConfig.from_pretrained(model_path[19:])
    base_model = AutoModelForCausalLM.from_pretrained("modelscope/Llama-2-7b-chat-ms",mirror="modelscope")
    model=PeftModel.from_pretrained(base_model,model_path[19:])
    print("Load model OK")
    return model, tokenizer

def process_input_file(input_file: str) -> pd.DataFrame:
    """Read the input file and return a pandas DataFrame"""
    return pd.read_csv(input_file, sep="\t", header=None, names=["content", "short_questions"], on_bad_lines='skip')

In [10]:
import traceback
def generate_code_completions(
    model, tokenizer, input_file: str, output_file: str, temperature: float
) -> None:
    """Generate code completions for the input file and save to the output file"""
    df = process_input_file(input_file)
    processed_df = pd.DataFrame(columns=["prompt", "output"])

    try:
        processed_df = pd.read_csv(output_file, sep="\t")
        last_index = processed_df.index[-1] if not processed_df.empty else -1
    except (FileNotFoundError, pd.errors.EmptyDataError):
#         print("Error of file")
        last_index = -1
    print(f"Total row:{last_index}")
    for i, row in df.iterrows():
        if i <= last_index:
            continue

        print("Row Number:", i)
#         prompt = f'Aşağıdaki materyali okuyun: {row["content"]}，Aşağıdaki soruları cevaplayın:{row["multiple_questions"]}'
        prompt = f'{row["content"]}{row["short_questions"]}'
        try:
            response = get_code_completion(prompt, model, tokenizer, temperature)
            print(f"Traversing index at: {i}")
            print(response)
            processed_df = processed_df.append({"prompt":prompt, "output": response}, ignore_index=True)
#             processed_df = processed_df.append({"content":row["content"],"question": row["multiple_questions"], "output": response}, ignore_index=True)

            if i % 10 == 0:
                processed_df.to_csv(output_file, sep="\t", index=False)

        except Exception as e:
#             print(len(str(e)))
            print(f"Error encountered at row {i}: {str(e)}. Waiting 2 minutes before retrying...")
            traceback.print_exc()
            time.sleep(120)

    processed_df.to_csv(output_file, sep="\t", index=False)
    print("Generation complete.")


In [ ]:
def main() -> None:
    """Main entry point"""
    temperature=0.2
    model_path = "Kamyar-zeinalipour/TR_QUIZ_GEN_MULTI_LLAMA7B"
    input_file = "Train-dataset.csv"
    output_file = "result.csv"
    ms.ms_memory_recycle()
    ms.set_context(memory_optimize_level='O0' )  # 启用内存优化
    model, tokenizer = load_model(model_path)
    ms.ms_memory_recycle()
    # ms.memory_reporter()
    generate_code_completions(model, tokenizer,input_file,output_file,temperature)

if __name__ == "__main__":
    main()

LlamaTokenizerFast(name_or_path='Kamyar-zeinalipour/TR_QUIZ_GEN_MULTI_LLAMA7B', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
[WARNING] DEVICE(1578,ffffb50110b0,python):2025-02-16-17:29:05.080.934 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:188] CheckVmmDriverVersion] Driver version is less than 24.0.0, vmm is disabled by default, drvier_version: 23.0.6
Loading checkpoint shards: 100%|██████████| 2/2 [01:14<00:00, 37.47s/it]
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9

Load model OK
Total row:-1
Row Number: 0
input_ids dtype: Int64
Traversing index at: 0
<s> content,multiple_questionsnan,nan,nan
You are a
Row Number: 1
input_ids dtype: Int64
Traversing index at: 1
<s> Standart olmayan şartlardaki hücre potansiyelidir. Aynı hücrenin standart şartlardaki hücre potansiyelidir.n:  Tepkimede alınan veya verilen elektron sayısıdır.Q: Tepkime oranıdır (Denge sabiti).
tepkimesinde bir pilin potansiyeli genel olarak
eşitliğiyle hesaplanır. Bu eşitlikte  ve   deney şartlarındaki   ve    iyonlarının derişimleridir.,"Question 1: Nernst Eşitliği hangi durumu hesaplamak için kullanılır?nan:  Teknolojik
Row Number: 2
input_ids dtype: Int64
Traversing index at: 2
<s> Choices: A. Standart olmayan şartlardaki hücre potansiyelini, B. Tepkime oranını, C. Elektron sayısınınan hücredeki enerji d
Row Number: 3
input_ids dtype: Int64
Traversing index at: 3
<s> Right Answer: Standart olmayan şartlardaki hücre potansiyelini"nanopatent" adını
Row Number: 4
input_ids dtype: Int

[ERROR] PRE_ACT(1578,ffff417ea1e0,python):2025-02-16-17:31:27.214.234 [mindspore/ccsrc/backend/common/mem_reuse/abstract_dynamic_mem_pool.cc:316] ExpandBlock] Expand block failed, expand size : 0.
Traceback (most recent call last):
  File "/tmp/ipykernel_1578/2841708995.py", line 24, in generate_code_completions
    response = get_code_completion(prompt, model, tokenizer, temperature)
  File "/tmp/ipykernel_1578/2579591544.py", line 14, in get_code_completion
    outputs = model.generate(
  File "/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/peft/peft_model.py", line 667, in generate
    outputs = self.base_model.generate(**kwargs)
  File "/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/core/utils/_contextlib.py", line 117, in decorate_context
    return func(*args, **kwargs)
  File "/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/generation/utils.py", line 2024, in generate
    result = sel

Error during code generation: Allocate memory failed

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ccsrc/runtime/device/device_address_utils.cc:1299 MallocForOutputs

Error encountered at row 4: Allocate memory failed

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ccsrc/runtime/device/device_address_utils.cc:1299 MallocForOutputs
. Waiting 2 minutes before retrying...
